## DATA WRANGLING - @WERATEDOGS DATASET
In this project,

In [89]:
# import installed packages
import pandas as pd
import numpy as np
import requests
import json
import tweepy
from tweepy import OAuthHandler
from timeit import default_timer as timer
from bs4 import BeautifulSoup

In [97]:
# setup tweepy
_consumer_key = ''
_consumer_secret = ''
_access_token = ''
_access_token_secret = ''

with open('auth_keys.txt', 'r') as auth_keys:
    try:
        _consumer_key = auth_keys.readline().split('"')[1:-1][0]
        _consumer_secret = auth_keys.readline().split('"')[1:-1][0]
        _access_token = auth_keys.readline().split('"')[1:-1][0]
        _access_token_secret = auth_keys.readline().split('"')[1:-1][0]
    except:
        raise Exception('Error: auth_keys.txt is missing or keys not found in source.')

auth = OAuthHandler(_consumer_key, _consumer_secret)
auth.set_access_token(_access_token, _access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [98]:
# reusable functions
def get_image_predictions_tsv():
    url = " https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
    response = requests.get(url)
    with open('image-predictions.tsv', 'wb') as file:
        file.write(response.content)
        
def get_tweet_status(tweet_id):
    if (isinstance(tweet_id, str)):
        try:
            tweet_status = api.get_status(tweet_id, tweet_mode='extended')
            return tweet_status # retweet_count | favorite_count | id_str
        except:
            return None
    else:
        raise Exception("Invalid argument types passed to function.")      

### 1. GATHERING DATA

In [4]:
# Gather data from memory
tweets_df = pd.read_csv("twitter-archive-enhanced.csv")
images_df = pd.read_csv("image-predictions.tsv", sep="\t")

In [6]:
# Get each tweet's status string using Tweepy 
with open('tweet_json.txt', 'w') as file:
        start = timer()
        for index, row in tweets_df.iterrows():
            tweet_status = get_tweet_status(str(row['tweet_id']))
            # Store each tweet's entire status string in tweet_json.txt file
            if tweet_status is None:
                file.write('id_str: u\'Invalid\',retweet_count\': Invalid, favorite_count\': Invalid,\n')
            file.write(str(tweet_status) + '\n')
        end = timer()

print('Time taken: ' + str(end - start)) # 47m 31.4s

# Create a DataFrame with tweet_id, retweet_count and favorite_count for each tweet
df_data = {'tweet_id': [], 'retweet_count': [], 'favorite_count': []}

with open('tweet_json.txt', 'r') as tweet_json:
    lines = tweet_json.read().split('\n');
    for line in lines:
        try:
           df_data['tweet_id'].append(line.split('id_str\': u\'')[1].split('\'')[0])
           df_data['retweet_count'].append(line.split('retweet_count\': ')[1].split(',')[0])
           df_data['favorite_count'].append(line.split('favorite_count\': ')[1].split(',')[0]) 
        except:
            pass

tweets_meta_df = pd.DataFrame(df_data)
tweets_meta_df.to_csv('tweets_meta.csv')

### 2. ASSESS

In [ ]:
tweets_df

In [ ]:
images_df

In [ ]:
tweets_meta_df

In [ ]:
tweets_df.info()

In [ ]:
images_df.info()

In [ ]:
tweets_meta_df.info()

In [7]:
all_columns = pd.Series(list(tweets_df) + list(images_df) + list(tweets_meta_df))
all_columns[all_columns.duplicated()]

17    tweet_id
31    tweet_id
dtype: object

In [8]:
tweets_df.head(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


### 3. CLEANING DATA

#### QUALITY ISSUES 
##### Twitter Archive Enhanced Dataset 
- `time_stamp column:` values appended with unecessary '+0000' || erronous data type - object
- `text_column`: missing substring of tweet url in some entries
- `source column:` source value still contains html
- `retweeted_status_user_id column:` id length is 10 chars, sometimes 9 chars
- `retweeted_status_timestamp column:` values appended with unecessary '+0000' || erronous data type - object
- `expanded_urls:` multiple urls in some entries || external (non-twitter & non-weratedogs) urls in some entries
- `rating_numerator column:` some values are way below 10 - a possible error based on the rating system
- `rating_denominator column:` some values are in the 20s to 100s - possible error based on rating system
- `rating_denominator column:` some entries don't have 10 - have 0, 70, 7, 11 ...
- `name column:`  'None' and single char non-name-like values for some entries || lower case names in som entries || language-specific accent symbols on some entries
##### Images Prediction Dataset
- `p1 column:` mixed captialized and lowercase values || words separeted by underscore instead of space
- `p1_conf column:` inconsistent format of numbers - variable decimal points 
- `p2 column`: mixed captialized and lowercase values || words separeted by underscore instead of space
- `p2_conf column:` inconsistent format of numbers - variable decimal points 
- `p3 column`: mixed captialized and lowercase values || words separeted by underscore instead of space
- `p3_conf column:` inconsistent format of numbers - variable decimal points 
- `missing values:` 2075 rows instead of 2356 rows
##### Tweets Meta Dataset
- `tweet_id column:` inconsistent id lengths in some entries
- `missing values:` 251 rows instead of 2356 rows

#### TIDINESS ISSUES
##### Twitter Archive Enhanced Dataset
- `text column:` should be split into text and tweet_url
- `tweets table && tweets_meta table:` merge two dataframes into one

In [2]:
# Make copies of the original data
clean_tweets = tweets_df.copy()
clean_images = images_df.copy()
clean_tweets_meta = tweets_meta_df.copy()

NameError: name 'tweets_df' is not defined

Multiple URLs in the `expanded_urls` column in the tweets_df. In addition to that, some urls are external non-twitter urls, and some entries are empty.

<b>Define</b><br />
Select on twitter url, if available or leave cell empty.

<b>Code</b>

In [ ]:
for index, row in clean_tweets.iterrows():
    if (clean_tweets.loc[index, 'expanded_urls']):
        links = clean_tweets.loc[2, 'expanded_urls'].split(',')
        for link in links:
            if ('https://twitter.com/' in link):
                clean_tweets.loc[index, 'expanded_urls'] = link
                break
            else:
                continue
    else:
        clean_tweets.loc[index, 'expanded_urls'] = np.NaN

<b>Test</b>

In [1]:
clean_tweets.expanded_urls.sample(100)

NameError: name 'clean_tweets' is not defined

Erroneous Data Type Object for `time stamp column` and the `retweeted_status_timestamp column` in the tweets_df and unecessary `+0000`

<b>Define</b><br />
Change the data types from Object to date and slice off the +0000 

<b>Code</b>

In [ ]:
for index, row in clean_tweets.iterrows():
    if ('+' in clean_tweets.loc[index, 'timestamp']):
        clean_tweets.loc[index, 'timestamp'] = row['timestamp'].split('+')[0]
    else:
        pass

for index, row in clean_tweets.iterrows():
    if ('+' in str(clean_tweets.loc[index, 'retweeted_status_timestamp'])):
        clean_tweets.loc[index, 'retweeted_status_timestamp'] = str(row['retweeted_status_timestamp']).split('+')[0]
    else:
        pass

clean_tweets['timestamp'] = pd.to_datetime(clean_tweets['timestamp'])

clean_tweets['retweeted_status_timestamp'] = pd.to_datetime(clean_tweets['retweeted_status_timestamp'])

<b>Test</b><br />

In [88]:
clean_tweets['timestamp'].sample(5)

2051   2015-12-01 00:38:31
1949   2015-12-07 02:25:23
675    2016-10-21 18:16:44
1174   2016-04-13 01:22:10
346    2017-02-15 18:03:45
Name: timestamp, dtype: datetime64[ns]

In [87]:
clean_tweets['retweeted_status_timestamp'].sample(5)

406    2016-10-06 15:49:14
1491                   NaT
1230                   NaT
759    2016-02-26 02:20:37
2061                   NaT
Name: retweeted_status_timestamp, dtype: datetime64[ns]

`Source column` in tweets_df contains html

<b>Define</b><br />
Strip out the html content from the value

<b>Code<b/>

In [ ]:
for index, row in clean_tweets.iterrows():
    if (clean_tweets.loc[index, 'source']):
        soup = BeautifulSoup(clean_tweets.loc[index, 'source'])
        clean_tweets.loc[index, 'source'] = soup.find('a').text
    else:
        pass

<b>Test</b>

In [109]:
clean_tweets.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
397,825120256414846976,NaN,NaN,2017-01-27 23:16:13 +0000,Twitter for iPhone,RT @dog_rates: Say hello to Pablo. He's one go...,8.250266e+17,4.196984e+09,2017-01-27 17:04:02 +0000,https://www.gofundme.com/my-puppys-double-cata...,12,10,Pablo,None,None,None,puppo
1629,684567543613382656,NaN,NaN,2016-01-06 02:49:55 +0000,Twitter for iPhone,This is Bobby. He doesn't give a damn about pe...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/684567543...,4,10,Bobby,None,None,None,None
867,761976711479193600,NaN,NaN,2016-08-06 17:26:19 +0000,Twitter for iPhone,This is Shelby. She finds stuff to put on her ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/761976711...,12,10,Shelby,None,None,None,None
2331,666353288456101888,NaN,NaN,2015-11-16 20:32:58 +0000,Twitter for iPhone,Here we have a mixed Asiago from the Galápagos...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666353288...,8,10,None,None,None,None,None
1069,740365076218183684,NaN,NaN,2016-06-08 02:09:24 +0000,Twitter for iPhone,When the photographer forgets to tell you wher...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/740365076...,10,10,None,None,None,None,None


<b>Tidiness</b>

Text column in `tweets table` contains two variables: text and tweet_url

<b>Define</b> <br />
Extract the <i>tweet_text</i> and <i>tweet_url</i> variables from the text column using regular expressions and the python `str.split()` method. Use an if-else conditional check when chcking url substring, since some rows are empty.

<b>Code</b>

In [111]:
tweet_text_list = []
tweet_url_list = []

for index, row in clean_tweets.iterrows():
    try:
        if ('https://' in clean_tweets.loc[index, 'text']):
            tweet_text_list.append(clean_tweets.loc[index, 'text'].split('https://')[0])
            tweet_url_list.append('https://' + clean_tweets.loc[index, 'text'].split('https://')[1])
        else:
            tweet_text_list.append('')
            tweet_url_list.append('')
    except:
        pass

clean_tweets['tweet_text'] = tweet_text_list
clean_tweets['tweet_url'] = tweet_url_list
clean_tweets = clean_tweets.drop('text', axis=1)

<b>Test</b>

In [112]:
clean_tweets.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,tweet_text,tweet_url
1321,706310011488698368,NaN,NaN,2016-03-06 02:46:44 +0000,Twitter for iPhone,NaN,NaN,NaN,https://twitter.com/dog_rates/status/706310011...,12,10,None,None,None,pupper,None,Here's a very sleepy pupper. Thinks it's an ai...,https://t.co/GGmcTIkBbf
2140,670003130994700288,NaN,NaN,2015-11-26 22:16:09 +0000,Twitter for iPhone,NaN,NaN,NaN,https://twitter.com/dog_rates/status/670003130...,10,10,Raphael,None,None,None,None,This is Raphael. He is a Baskerville Conquista...,https://t.co/3NTykJmtHt
457,818145370475810820,NaN,NaN,2017-01-08 17:20:31 +0000,Twitter for iPhone,NaN,NaN,NaN,https://twitter.com/dog_rates/status/818145370...,11,10,Autumn,None,None,None,None,This is Autumn. Her favorite toy is a cheesebu...,https://t.co/JlPug12E5Z
629,794355576146903043,NaN,NaN,2016-11-04 01:48:22 +0000,Twitter for iPhone,7.887659e+17,4.196984e+09,2016-10-19 15:37:03 +0000,https://twitter.com/dog_rates/status/788765914...,12,10,Butter,None,None,None,None,RT @dog_rates: This is Butter. She can have wh...,https://t.co/x5gXRS1abq
314,835172783151792128,NaN,NaN,2017-02-24 17:01:22 +0000,Twitter for iPhone,NaN,NaN,NaN,https://twitter.com/dog_rates/status/835172783...,12,10,None,None,None,None,None,We only rate dogs. Please don't send in any no...,https://t.co/8fX2VkExnL
